### Задача 1.

Примените оконные функции к таблице `products` и с помощью ранжирующих функций упорядочьте все товары по цене — от самых дорогих к самым дешёвым. Добавьте в таблицу следующие колонки:

- Колонку `product_number` с порядковым номером товара (функция `ROW_NUMBER`).
- Колонку `product_rank` с рангом товара с пропусками рангов (функция `RANK`).
- Колонку `product_dense_rank` с рангом товара без пропусков рангов (функция `DENSE_RANK`).

Не забывайте указывать в окне сортировку записей — без неё ранжирующие функции могут давать некорректный результат, если таблица заранее не отсортирована. Деление на партиции внутри окна сейчас не требуется. Сортировать записи в результирующей таблице тоже не нужно.

Поля в результирующей таблице: `product_id`, `name`, `price`, `product_number`, `product_rank`, `product_dense_rank`

```SQL
SELECT 
    product_id,
    name,
    price,
    ROW_NUMBER() OVER(ORDER BY price DESC) AS product_number,
    RANK() OVER(ORDER BY price DESC) AS product_rank,
    DENSE_RANK() OVER(ORDER BY price DESC) AS product_dense_rank
FROM products
```

### Задача 2.

Примените оконную функцию к таблице `products` и с помощью агрегирующей функции в отдельной колонке для каждой записи проставьте цену самого дорогого товара. Колонку с этим значением назовите `max_price`. Затем для каждого товара посчитайте долю его цены в стоимости самого дорогого товара — просто поделите одну колонку на другую. Полученные доли округлите до двух знаков после запятой. Колонку с долями назовите `share_of_max`.

Выведите всю информацию о товарах, включая значения в новых колонках. Результат отсортируйте сначала по убыванию цены товара, затем по возрастанию `id` товара.

Поля в результирующей таблице: `product_id`, `name`, `price`, `max_price`, `share_of_max`

```SQL
SELECT
    product_id,
    name,
    price,
    MAX(price) OVER() AS max_price,
    ROUND(price / MAX(price) OVER(), 2) AS share_of_max
FROM products
ORDER BY price DESC, product_id ASC
```

### Задача 3.

Примените две оконные функции к таблице `products` — одну с агрегирующей функцией `MAX`, а другую с агрегирующей функцией `MIN` — для вычисления максимальной и минимальной цены. Для двух окон задайте инструкцию `ORDER BY` по убыванию цены. Поместите результат вычислений в две колонки `max_price` и `min_price`.

Выведите всю информацию о товарах, включая значения в новых колонках. Результат отсортируйте сначала по убыванию цены товара, затем по возрастанию `id` товара.

Поля в результирующей таблице: `product_id`, `name`, `price`, `max_price`, `min_price`

```SQL
SELECT
    product_id,
    name,
    price,
    MAX(price) OVER(ORDER BY price DESC) AS max_price,
    MIN(price) OVER(ORDER BY price DESC) AS min_price
FROM products
ORDER BY price DESC, product_id ASC
```

### Задача 4.

Сначала на основе таблицы `orders` сформируйте новую таблицу с общим числом заказов по дням. При подсчёте числа заказов не учитывайте отменённые заказы (их можно определить по таблице `user_actions`). Колонку с днями назовите `date`, а колонку с числом заказов — `orders_count`.

Затем поместите полученную таблицу в подзапрос и примените к ней оконную функцию в паре с агрегирующей функцией `SUM` для расчёта накопительной суммы числа заказов. Не забудьте для окна задать инструкцию `ORDER BY` по дате.

Колонку с накопительной суммой назовите `orders_cum_count`. В результате такой операции значение накопительной суммы для последнего дня должно получиться равным общему числу заказов за весь период.

Сортировку результирующей таблицы делать не нужно.

Поля в результирующей таблице: `date`, `orders_count`, `orders_cum_count`

```SQL
SELECT 
    date,
    orders_count,
    SUM(orders_count) OVER(ORDER BY date)::INTEGER AS orders_cum_count
FROM (
    SELECT
        creation_time::DATE AS date,
        COUNT(DISTINCT order_id) AS orders_count
    FROM orders
    WHERE order_id NOT IN (SELECT order_id FROM user_actions  WHERE action = 'cancel_order')
    GROUP BY creation_time::DATE 
    ORDER BY creation_time::DATE ASC) t
```

### Задача 5.

Для каждого пользователя в таблице `user_actions` посчитайте порядковый номер каждого заказа. Для этого примените оконную функцию `ROW_NUMBER` к колонке с временем заказа. Не забудьте указать деление на партиции по пользователям и сортировку внутри партиций. Отменённые заказы не учитывайте. Новую колонку с порядковым номером заказа назовите `order_number`. Результат отсортируйте сначала по возрастанию id пользователя, затем по возрастанию `order_number`. Добавьте `LIMIT 1000`.

Поля в результирующей таблице: `user_id`, `order_id`, `time`, `order_number`

```SQL
SELECT 
    user_id,
    order_id, 
    time,
    ROW_NUMBER() OVER(PARTITION BY user_id ORDER BY time) AS order_number
FROM user_actions
WHERE order_id NOT IN (SELECT order_id FROM user_actions  WHERE action = 'cancel_order')
ORDER BY user_id, order_number
LIMIT 1000
```

### Задача 6.

Дополните запрос из предыдущего задания и с помощью оконной функции для каждого заказа каждого пользователя рассчитайте, сколько времени прошло с момента предыдущего заказа. 

Для этого сначала в отдельном столбце с помощью `LAG` сделайте смещение по столбцу `time` на одно значение назад. Столбец со смещёнными значениями назовите `time_lag`. Затем отнимите от каждого значения в колонке `time` новое значение со смещением (либо можете использовать уже знакомую функцию `AGE`). Колонку с полученным интервалом назовите `time_diff`. Менять формат отображения значений не нужно, они должны иметь примерно следующий вид:

>```SQL
3 days, 12:18:22
>```

По-прежнему не учитывайте отменённые заказы. Также оставьте в запросе порядковый номер каждого заказа, рассчитанный на прошлом шаге. Результат отсортируйте сначала по возрастанию id пользователя, затем по возрастанию порядкового номера заказа. Добавьте `LIMIT 1000`.

Поля в результирующей таблице: `user_id`, `order_id`, `time`, `order_number`, `time_lag`, `time_diff`

```SQL
SELECT 
    user_id,
    order_id, 
    time,
    ROW_NUMBER() OVER(PARTITION BY user_id ORDER BY time) AS order_number,
    LAG(time) OVER(PARTITION BY user_id ORDER BY time) AS time_lag,
    AGE(time, LAG(time) OVER(PARTITION BY user_id ORDER BY time)) AS time_diff
FROM user_actions
WHERE order_id NOT IN (SELECT order_id FROM user_actions  WHERE action = 'cancel_order')
ORDER BY user_id, order_number
LIMIT 1000
```

### Задача 7.

На основе запроса из предыдущего задания для каждого пользователя рассчитайте, сколько в среднем времени проходит между его заказами. Посчитайте этот показатель только для тех пользователей, которые за всё время оформили более одного неотмененного заказа.

Среднее время между заказами выразите в часах, округлив значения до целого числа. Колонку со средним значением времени назовите `hours_between_orders`. Результат отсортируйте по возрастанию id пользователя.

Добавьте в запрос оператор `LIMIT` и включите в результат только первые 1000 записей.

Поля в результирующей таблице: `user_id`, `hours_between_orders`

```SQL

```